In [3]:
import os
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy.spatial.transform import Rotation as R
import time
import pickle
import argparse
from utils import *

from deepVCP import DeepVCP
from CustomDataset2 import CustomDataset2
from deepVCP_loss import deepVCP_loss
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
dataset = 'CustomDataset2'
retrain_path = 'store'
model_path = 'Optimise_final_model.pt'
full_dataset = True

In [3]:
num_epochs = 20
batch_size = 1
lr = 0.001
# loss balancing factor 
alpha = 0.5

print(f"Params: epochs: {num_epochs}, batch: {batch_size}, lr: {lr}, alpha: {alpha}\n")

# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")


Params: epochs: 20, batch: 1, lr: 0.001, alpha: 0.5

device: cuda


In [4]:
root = 'meshes/'
train_data= CustomDataset2(root=root)

Processing bun000_v2.ply
(40245, 3)
Processing bun045_v2.ply
(40091, 3)
Processing bun090_v2.ply
(30373, 3)
Processing bun180_v2.ply
(40247, 3)
Processing bun270_v2.ply
(31697, 3)
Processing bun315_v2.ply
(35334, 3)
# Total clouds 6


In [5]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)

for n_batch, (src, target, R_gt, t_gt, ) in enumerate(train_loader):
    print(src.shape, target.shape)
    print(R_gt, t_gt)
    break


num_train = len(train_data)
print('Train dataset size: ', num_train)

use_normal = dataset == "modelnet"

# Initialize the model
model = DeepVCP(use_normal=use_normal)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

torch.Size([1, 3, 10000]) torch.Size([1, 3, 10000])
tensor([[[ 0.7743, -0.3514,  0.5262],
         [ 0.4306,  0.9020, -0.0313],
         [-0.4636,  0.2509,  0.8498]]], dtype=torch.float64) tensor([[[0.0742],
         [0.0742],
         [0.0742]]], dtype=torch.float64)
Train dataset size:  30


DeepVCP(
  (FE1): feat_extraction_layer(
    (sa1): PointNetSetAbstraction(
      (mlp_convs): ModuleList(
        (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (mlp_bns): ModuleList(
        (0-1): 2 x BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (sa2): PointNetSetAbstraction(
      (mlp_convs): ModuleList(
        (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (mlp_bns): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (sa3): PointNetSetAbstraction(

In [6]:
optim = Adam(model.parameters(), lr=lr)

# begin train 
model.train()
loss_epoch_avg = []
for epoch in tqdm(range(num_epochs)):
    print(f"epoch #{epoch}")
    loss_epoch = []
    running_loss = 0.0

    for n_batch, (src, target, R_gt, t_gt, ) in enumerate(train_loader):
        start_time = time.time()
        # mini batch
        src, target, R_gt, t_gt = src.to(device), target.to(device), R_gt.to(device), t_gt.to(device)
        t_init = torch.randn((1, 3))
        R_prior = R_gt.clone()
        corruption = torch.FloatTensor(batch_size, 3, 3).uniform_(0.9, 1.1).cuda()
        R_prior = R_prior*corruption
        src_keypts, target_vcp = model(src, target, R_prior, t_init)
        # print('src_keypts shape', src_keypts.shape)
        # print('target_vcp shape', target_vcp.shape)
        # zero gradient 
        optim.zero_grad()
        loss, R_pred, t_pred = deepVCP_loss(src_keypts, target_vcp, R_gt, t_gt, alpha=0.5)


        # error metric for rigid body transformation
        r_pred = R.from_matrix(R_pred.squeeze(0).cpu().detach().numpy())
        r_pred_arr = torch.tensor(r_pred.as_euler('xyz', degrees=True)).reshape(1, 3)
        r_gt = R.from_matrix(R_gt.squeeze(0).cpu().detach().numpy())
        r_gt_arr = torch.tensor(r_gt.as_euler('xyz', degrees=True)).reshape(1, 3)
        pdist = nn.PairwiseDistance(p = 2)
        t_pred = t_pred.squeeze(-1)
        t_gt = t_gt.squeeze(-1)

        print("rotation error: ", pdist(r_pred_arr, r_gt_arr).mean())
        print("translation error: ", pdist(t_pred, t_gt).mean())

        # backward pass
        loss.backward()
        # update parameters 
        optim.step()

        running_loss += loss.item()
        loss_epoch += [loss.item()]
        print("--- %s seconds ---" % (time.time() - start_time))
        if (n_batch + 1) % 5 == 0:
            print("Epoch: [{}/{}], Batch: {}, Loss: {}".format(
                epoch, num_epochs, n_batch, loss.item()))
            running_loss = 0.0

    torch.save(model.state_dict(), "epoch_" + str(epoch) + "_model.pt")
    loss_epoch_avg += [sum(loss_epoch) / len(loss_epoch)]
    print("loss epoch", loss_epoch)
    with open("training_loss_" + str(epoch) + ".txt", "wb") as fp:   #Pickling
        pickle.dump(loss_epoch, fp)


  0%|          | 0/20 [00:00<?, ?it/s]

epoch #0
feature extraction time:  7.882020473480225
src_keypts_idx_unsqueezed:  torch.Size([1, 3, 64])
src_keypts:  torch.Size([1, 64, 3])
Grouping keypoints time:  0.013999700546264648
B:  1
K_topk:  64
nsample:  32
num_feat:  32
get_cat_feat_src time:  0.0


c:\Users\bingwang\.conda\envs\InverseCW4\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([9579, 8410,  355,  ..., 5979, 3797, 3177], device='cuda:0')
get_cat_feat_tgt time:  0.100982666015625
Loss: 0.27637573912479096
rotation error:  tensor(3.1098, dtype=torch.float64)
translation error:  tensor(0.4787, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
--- 15.347990989685059 seconds ---
feature extraction time:  6.659988164901733
src_keypts_idx_unsqueezed:  torch.Size([1, 3, 64])
src_keypts:  torch.Size([1, 64, 3])
Grouping keypoints time:  0.021999835968017578
B:  1
K_topk:  64
nsample:  32
num_feat:  32
get_cat_feat_src time:  0.0
tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  ten

  5%|▌         | 1/20 [06:38<2:06:17, 398.82s/it]

...

tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([6966, 4313, 1675,  ..., 8593, 9829, 6680], device='cuda:0')
get_cat_feat_tgt time:  0.11700034141540527
Loss: 0.0072778938010690485
rotation error:  tensor(2.3618, dtype=torch.float64)
translation error:  tensor(0.0171, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
--- 13.82204532623291 seconds ---
Epoch: [17/20], Batch: 29, Loss: 0.0072778938010690485
loss epoch [0.007101844453778752, 0.0053059041301837424, 0.003577665635534089, 0.008198778137431029, 0.01025476023662067, 0.00899944269687792, 0.004555304080557743, 0.0076846761745672666, 0.010440039772362405, 0.009368126965572158, 0.013893098527825474, 0.009010420561295718, 0.0019767683554266753, 0.014755546026550792, 0.020468534645466267, 0.021674740556916147, 0.01879588804

 95%|█████████▌| 19/20 [2:09:09<06:51, 411.25s/it]

tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([3514, 2237, 1997,  ..., 1010,   11, 3676], device='cuda:0')
get_cat_feat_tgt time:  0.10899996757507324
Loss: 0.00913322552420936
rotation error:  tensor(2.6720, dtype=torch.float64)
translation error:  tensor(0.0153, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
--- 14.24802541732788 seconds ---
Epoch: [18/20], Batch: 29, Loss: 0.00913322552420936
loss epoch [0.004158405221119931, 0.008284845365978238, 0.006362304733642519, 0.008818809417136093, 0.005600680331517733, 0.008279658196878712, 0.005996144963810844, 0.007752921791168706, 0.006128740131422974, 0.005819107442149906, 0.010086421061762555, 0.007461287455531037, 0.00487284186529633, 0.0050385199816559145, 0.006164372838116749, 0.005391919337560261, 0.0050891567244544

100%|██████████| 20/20 [2:16:03<00:00, 408.16s/it]

tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([ 733, 7868, 3823,  ..., 9569, 8071, 4259], device='cuda:0')
get_cat_feat_tgt time:  0.1100003719329834
Loss: 0.010870042277999254
rotation error:  tensor(2.3333, dtype=torch.float64)
translation error:  tensor(0.0183, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
--- 13.630985260009766 seconds ---
Epoch: [19/20], Batch: 29, Loss: 0.010870042277999254
loss epoch [0.0024786231472144864, 0.003632966313616842, 0.002870603620226793, 0.0026043047959128326, 0.004516337857406418, 0.00420837519842149, 0.0027043296125505377, 0.004761109324859243, 0.006086536667981743, 0.007174659715140915, 0.00720774022177792, 0.002703543520521084, 0.0011873903168894318, 0.0034071105618079787, 0.004332734126124681, 0.0048197837961049695, 0.0041780059

In [ ]:
# save
print("Finished Training")
torch.save(model.state_dict(), model_path)

Finished Training
